*QUESTION01*

In [15]:
from collections import defaultdict

def is_all_packages_assigned(package_assignment, package_list):
    return len(package_assignment) == len(package_list)

def is_valid_package_assignment(package_assignment, city_capacity_limit):
    city_package_count = defaultdict(int)
    for package, city in package_assignment.items():
        city_package_count[city] += 1
        if city_package_count[city] > city_capacity_limit[city]:
            return False
    return True

def apply_forward_checking(package_assignment, available_cities, city_capacity_limit):
    new_city_options = {pkg: list(available_cities[pkg]) for pkg in available_cities if pkg not in package_assignment}
    city_package_count = defaultdict(int)

    for package, city in package_assignment.items():
        city_package_count[city] += 1

    for package in new_city_options:
        new_city_options[package] = [city for city in new_city_options[package] if city_package_count[city] < city_capacity_limit[city]]
        if not new_city_options[package]:
            return None  # No valid city left for a package, need to backtrack

    return new_city_options

def find_package_assignment(package_assignment, available_cities, city_capacity_limit, package_list):
    if is_all_packages_assigned(package_assignment, package_list):
        return package_assignment

    next_package = next(pkg for pkg in package_list if pkg not in package_assignment)  # Pick first unassigned package
    for city in available_cities[next_package]:
        package_assignment[next_package] = city
        print(f"Package {next_package} -> {city}", end="")

        updated_cities = apply_forward_checking(package_assignment, available_cities, city_capacity_limit)

        if updated_cities is not None and is_valid_package_assignment(package_assignment, city_capacity_limit):
            if len(available_cities[next_package]) == 1:
                print("  (Only one option available, assigned first)")
            else:
                print("  (Forward Checking applied)")
            result = find_package_assignment(package_assignment, updated_cities, city_capacity_limit, package_list)
            if result:
                return result

        del package_assignment[next_package]  # Backtrack

    return None

def solve_delivery_problem():
    package_list = [1, 2, 3, 4]
    city_list = ['A', 'B', 'C']
    city_capacity_limit = {'A': 2, 'B': 3, 'C': 1}
    package_allowed_cities = {
        1: ['A', 'B'],
        2: ['B', 'C'],
        3: ['A', 'C'],
        4: ['A']
    }

    print("Package Assignments:")

    package_assignment = {}
    available_cities = {pkg: list(package_allowed_cities[pkg]) for pkg in package_list}

    return find_package_assignment(package_assignment, available_cities, city_capacity_limit, package_list)

solution = solve_delivery_problem()
print("\nOptimal Assignment:", solution)


Package Assignments:
Package 1 -> A  (Forward Checking applied)
Package 2 -> B  (Forward Checking applied)
Package 3 -> APackage 3 -> C  (Forward Checking applied)
Package 4 -> A  (Only one option available, assigned first)

Optimal Assignment: {1: 'A', 2: 'B', 3: 'C', 4: 'A'}


*QUESTION02*

In [25]:
from collections import deque

def ac3(domains, constraints, neighbors):
    queue = deque(constraints)
    reductions = []
    while queue:
        (xi, xj) = queue.popleft()
        if revise(domains, xi, xj, reductions):
            if not domains[xi]:  # If domain is empty, failure
                return False, reductions
            for xk in neighbors[xi]:
                if xk != xj:
                    queue.append((xk, xi))
    return True, reductions

def revise(domains, xi, xj, reductions):
    revised = False
    for x in set(domains[xi]):  # Iterate over a copy to allow modifications
        if all(x == y for y in domains[xj]):
            domains[xi].remove(x)
            reductions.append(f"- If {xi} = {x}, {xj} cannot be {x}.")
            revised = True
    return revised

def backtracking(assignment, domains, neighbors):
    if len(assignment) == len(domains):
        return assignment
    var = select_unassigned_variable(assignment, domains)
    for value in domains[var]:
        if is_consistent(var, value, assignment, neighbors):
            assignment[var] = value
            result = backtracking(assignment, domains, neighbors)
            if result:
                return result
            assignment.pop(var)
    return None

def select_unassigned_variable(assignment, domains):
    return next(var for var in domains if var not in assignment)

def is_consistent(var, value, assignment, neighbors):
    return all(assignment.get(neigh) != value for neigh in neighbors[var])

regions = ['A', 'B', 'C', 'D']
colors = ['Red', 'Green', 'Blue']

neighbors = {
    'A': ['B', 'C'],
    'B': ['A', 'D'],
    'C': ['A', 'D'],
    'D': ['B', 'C']
}

domains = {region: set(colors) for region in regions}
constraints = [(xi, xj) for xi in neighbors for xj in neighbors[xi]]

ac3_success, reductions = ac3(domains, constraints, neighbors)

if ac3_success:
    print("Domain Reduction:")
    for reduction in reductions:
        print(reduction)
    
    solution = backtracking({}, domains, neighbors)
    if solution:
        print("\nRegion Assignments:")
        for region, color in solution.items():
            if len(domains[region]) == 1:
                print(f"{region} -> {color}  (Only valid choice after AC-3)")
            else:
                print(f"{region} -> {color}")
    else:
        print("No solution found.")
else:
    print("AC-3 failed: No solution possible.")

Domain Reduction:

Region Assignments:
A -> Red
B -> Green
C -> Green
D -> Red
